<a href="https://colab.research.google.com/github/HYUNSOOLEE-6839/colab-Deep-Learning/blob/main/NewYorkTimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM을 이용한 텍스트 생성
- 캐글 New York Times Comments
- ArticlesApril2018.csv

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv(filename, encoding='latin1')

In [4]:
df

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Torontoâs ...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothinâ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In âSomething Wonderful,â Todd S. Purdum a...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...


### 데이터 전처리

In [5]:
print('열의 개수', len(df.columns))
print(df.columns)

열의 개수 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
# headline 열의 데이터만 사용, null 검사
df['headline'].isnull().values.any()

False

In [7]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in df.headline if title != 'Unknown']
len(headline)

1214

In [9]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
# 구두점 제거와 소문자화를 위한 함수 생성
def repreprocessing(s):
  s=s.encode("utf8").decode("ascii",'ignore')
  return ''.join(c for c in s if c not in punctuation).lower()

In [11]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [12]:
# 단어 집합(voca)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) +1
print('단어 집합의 크기 :%d' % vocab_size)

단어 집합의 크기 :3494


In [13]:
sequences = []
for line in headline: # 1214개의 샘플에 대해서 샘플을 1개씩 가져온다.
  encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 2237],
 [99, 2237, 1115],
 [99, 2237, 1115, 582],
 [99, 2237, 1115, 582, 52],
 [99, 2237, 1115, 582, 52, 7],
 [99, 2237, 1115, 582, 52, 7, 2],
 [99, 2237, 1115, 582, 52, 7, 2, 372],
 [99, 2237, 1115, 582, 52, 7, 2, 372, 10],
 [99, 2237, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [1145, 2],
 [1145, 2, 3]]

In [14]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to 사용
  index_to_word[value] = key

print('빈도수 상위 1번 단어:', index_to_word[1]) 
print('빈도수 상위 582번 단어:', index_to_word[582]) 

빈도수 상위 1번 단어: the
빈도수 상위 582번 단어: offer


In [15]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이:', max_len)

샘플의 최대 길이: 23


In [16]:
# 전체 샘플의 길이를 23(가장 긴 샘플의 길이)로 패딩
# pre 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99 2237]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99 2237 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99 2237 1115  582]]


In [17]:
# 각 샘플의 마지막 단어를 레이블로 분리
X = sequences[:, :-1]
y = sequences[:, -1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [18]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)

In [19]:
X.shape, Y.shape, y.shape

((7385, 22), (7385, 3494), (7385,))

### 모델 정의

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [21]:
# 각 단어의 임베딩 벡터는 10차원, LSTM 노드 크기는 128
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128, name="LSTM_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
231/231 - 9s - loss: 7.5616 - accuracy: 0.0294
Epoch 2/200
231/231 - 1s - loss: 7.0068 - accuracy: 0.0305
Epoch 3/200
231/231 - 1s - loss: 6.8801 - accuracy: 0.0343
Epoch 4/200
231/231 - 1s - loss: 6.7803 - accuracy: 0.0383
Epoch 5/200
231/231 - 1s - loss: 6.6650 - accuracy: 0.0399
Epoch 6/200
231/231 - 1s - loss: 6.5287 - accuracy: 0.0460
Epoch 7/200
231/231 - 1s - loss: 6.3896 - accuracy: 0.0477
Epoch 8/200
231/231 - 1s - loss: 6.2386 - accuracy: 0.0539
Epoch 9/200
231/231 - 1s - loss: 6.0757 - accuracy: 0.0565
Epoch 10/200
231/231 - 1s - loss: 5.9188 - accuracy: 0.0604
Epoch 11/200
231/231 - 1s - loss: 5.7679 - accuracy: 0.0653
Epoch 12/200
231/231 - 1s - loss: 5.6251 - accuracy: 0.0697
Epoch 13/200
231/231 - 1s - loss: 5.4830 - accuracy: 0.0733
Epoch 14/200
231/231 - 1s - loss: 5.3493 - accuracy: 0.0753
Epoch 15/200
231/231 - 1s - loss: 5.2200 - accuracy: 0.0838
Epoch 16/200
231/231 - 1s - loss: 5.0922 - accuracy: 0.0877
Epoch 17/200
231/231 - 1s - loss: 4.9733 - accura

### 모델 검증

In [28]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n):
  init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
  sentence =''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
    encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
    result = np.argmax(model.predict(encoded), axis=-1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
    for word, index in t.word_index.items():
      if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측 단어이므로 break
    current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

  # for문이므로 이 행동을 다시 반복
  sentence = init_word + sentence
  return sentence

In [31]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'what', 10))

i cleaning people is consequences scientists baby come about the birth
how is out over cede night just on political for late
what drake drake has a limits of a uprising improve a
